In [17]:
#from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright
import json
from pathlib import Path
import time
from datetime import date, timedelta

In [18]:
# login info 
user, pw = 'katrina.mullan@mso.umt.edu', 'wvp6rau6rqb_bwy1EYT!'
url = 'https://pinnacle.azira.com/'

# start and end dates for data downloads
start_date = '2022-01-01'
end_date = date.today() - timedelta(days=7)
end_date = end_date.strftime("%Y-%m-%d")

In [19]:
end_date

'2025-12-12'

### Initialize (open page)

In [96]:
# open page - persistent across cells
p = await async_playwright().start()
browser = await p.chromium.launch(headless=False)

context = await browser.new_context(
        geolocation={"latitude": 46.8721, "longitude": -113.9940},  # Missoula-ish; pick whatever
        permissions=["geolocation"],
    )

page = await context.new_page()
page2 = await context.new_page()
#page2 = await context.new_page()

# single page version
# page = await browser.new_page()

### Navigate / execute page tests

In [91]:
await page.goto(url)

<Response url='https://pinnacle.azira.com/' request=<Request url='https://pinnacle.azira.com/' method='GET'>>

Login  |  https://pinnacle.azira.com/

In [38]:
await page.get_by_role("textbox", name="Email Address").fill(user)
await page.get_by_role("textbox", name="Password").fill(pw)
await page.get_by_role("button", name="Log in").click()

Home  |  https://pinnacle.azira.com/home/feeds/datasets

In [39]:
await page.get_by_role("button", name="Create New").click()

Format  |  https://pinnacle.azira.com/home/create/dataset

In [40]:
await page.get_by_role("button", name="Next").click()

Places |  https://pinnacle.azira.com/home/create/dataset/places

In [ ]:
# click "GEoJSON/KML/ESRI"
await page.get_by_role("heading", name="GeoJSON/KML/ESRI").click()

# paste the GeoJSON data
json_path = Path('/home/vince/Documents/SmartFires/osm_fitness/Wyoming_700k(6)/Casper.geojson')
json_text = json_path.read_text()

await page.get_by_role("textbox", name="Enter GeoJSON").fill(json_text)

await page.get_by_role("button", name="Next").click() 

'{\n"type": "FeatureCollection",\n"name": "Casper",\n"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },\n"features": [\n{ "type": "Feature", "properties": { "largest_city": "Casper", "all_cities": "Casper", "total_area_ft2": 12441893.223271275 }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -106.3369773, 42.7987525 ], [ -106.3369444, 42.7964198 ], [ -106.338241, 42.7964176 ], [ -106.3382934, 42.796581499999988 ], [ -106.338361, 42.796698599999978 ], [ -106.3384578, 42.79683150000001 ], [ -106.3391936, 42.797714500000019 ], [ -106.3394287, 42.79800059999998 ], [ -106.3386229, 42.7983565 ], [ -106.3383891, 42.7984807 ], [ -106.3382589, 42.7985331 ], [ -106.3381298, 42.7985829 ], [ -106.3378507, 42.7986583 ], [ -106.3375464, 42.798711 ], [ -106.3374733, 42.7987103 ], [ -106.3369773, 42.7987525 ] ] ], [ [ [ -106.348140078946017, 42.809487649267517 ], [ -106.347441911002136, 42.809501152579145 ], [ -106.347552134287525, 42.809173709950656 ]

Time Frame |  https://pinnacle.azira.com/home/create/dataset/timeframe

In [ ]:
await page.locator("#start-date-input").fill(start_date)
await page.locator("#end-date-input").fill(end_date)
await page.get_by_role("button", name="Next").click()



Insights 

In [ ]:
await page.get_by_text("Pin Dataset").click() 
await page.get_by_text("Common Evening and Daytime").click()   
await page.get_by_role("button", name="Next").click()

Details

In [ ]:
city = json_path.stem
state = json_path.parent.stem.split('_')[0]
name = f"{city}_{state}"

await page.get_by_role("textbox", name="Name is required").fill(name)

await page.get_by_role("button", name="Submit").first.click()


'Casper_Wyoming'

### Inspect pages

In [114]:
await page.pause()

get current url

### Teardown

In [ ]:
await browser.close()
await p.stop()

# Combined Script

### 1. Run First (register needed functons)

In [9]:
async def start_browser():
    playwright = await async_playwright().start()

    browser = await playwright.chromium.launch(headless=False)

    context = await browser.new_context(
        geolocation={"latitude": 46.8721, "longitude": -113.9940},
        permissions=["geolocation"],
    )

    page = await context.new_page()

    return {
        "playwright": playwright,
        "browser": browser,
        "context": context,
        "page": page,
    }

async def login(page, user, pw):
    await page.goto(url) 
    await page.get_by_role("textbox", name="Email Address").fill(user)
    await page.get_by_role("textbox", name="Password").fill(pw)
    await page.get_by_role("button", name="Log in").click()

## 2. Bulk Download

Get list of files to download

In [7]:
download_list = []

base_dir = Path('/home/vince/Documents/SmartFires/osm_fitness')
folders = [p for p in base_dir.iterdir() if p.is_dir()]

# ----REMOVE NEXT LINE TO DO ALL STATES-----
folders = [f for f in folders if 'Colorado_700k' in f.name]

for folder in folders:  
    geojson_files = list(folder.glob('*.geojson'))
    for json_path in geojson_files:
        download_list.append(json_path)  

download_list = download_list[0:1]

Download

In [ ]:
# login 
browser_handle = await start_browser()
page = browser_handle["page"]
await login(page, user, pw)

completed = []
failed = []

for i, json_file in enumerate(download_list):
    try:
        json_path = Path(json_file)
        json_text = json_file.read_text() # paste the GeoJSON data

        # for use on the last page before submitting the job
        city = json_path.stem.replace(' ', '_')
        state = json_path.parent.stem.split('_')[0].replace(' ', '_')
        project_name = f"{city}_{state}"

        if i == 0:
            time.sleep(3)
        await page.goto('https://pinnacle.azira.com/home/feeds/datasets') # go to main page

        await page.get_by_role("button", name="Create New").click()  # "Create New"
        await page.get_by_role("button", name="Next").click()  

        await page.get_by_role("heading", name="GeoJSON/KML/ESRI").click()   # click "GEoJSON/KML/ESRI"
        await page.get_by_role("textbox", name="Enter GeoJSON").fill(json_text)
        await page.get_by_role("button", name="Next").click() 

        await page.locator("#start-date-input").fill(start_date)
        await page.locator("#end-date-input").fill(end_date)
        await page.get_by_role("button", name="Next").click()

        await page.get_by_text("Pin Dataset").click() 
        await page.get_by_text("Common Evening and Daytime").click()   
        await page.get_by_role("button", name="Next").first.click() 

        await page.get_by_role("textbox", name="Name is required").fill(project_name)
        await page.get_by_role("button", name="Submit").first.click()

        # continue if "file too large"  warning appears  
        await page.wait_for_load_state("networkidle", timeout=10_000)

        completed.append(json_file) 
    except Exception as e:
        print(f"Error processing {json_file}: {e}")     
        failed.append(json_file)
